In [1]:
from keras.utils import to_categorical
from keras.layers import Dropout
from keras.layers import Dense
from keras import models
from keras import optimizers
from keras import backend as K
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import math
import gensim
from gensim.models import Word2Vec 
import time

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# ANN Metrics

In [2]:
def recall(y_true, y_pred):
   
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
    
def precision(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    
    myPrecision = precision(y_true, y_pred)
    myRecall = recall(y_true, y_pred)
    return 2*((myPrecision*myRecall)/(myPrecision+myRecall+K.epsilon()))

# ANN Model

In [18]:
tests[tests.label == 1]

,ts,src_ip,dest_port,duration,src_bytes,dest_bytes,conn_state,history,src_pkts,dest_pkts,local,label,location
13,1.547325e+09,157.230.3.234,23,-,-,-,S0,S,1,0,0,1,US
278,1.547338e+09,125.64.94.201,7776,-,-,-,S0,S,1,0,0,1,China
282,1.547345e+09,185.255.31.6,3355,-,-,-,S0,S,1,0,0,1,Russia
349,1.547283e+09,219.132.153.169,1433,-,-,-,S0,S,1,0,0,1,China
436,1.547307e+09,120.52.152.16,902,-,-,-,S0,S,1,0,0,1,China
469,1.547280e+09,185.176.27.106,7788,-,-,-,S0,S,1,0,0,1,Russia
473,1.547305e+09,115.233.246.46,22,-,-,-,S0,S,1,0,0,1,China
595,1.547322e+09,71.6.232.4,8443,-,-,-,S0,S,1,0,0,1,US
718,1.547346e+09,125.64.94.200,2160,-,-,-,S0,S,1,0,0,1,China
731,1.547307e+09,68.183.31.204,23,-,-,-,S0,S,1,0,0,1,US


In [3]:
df = pd.read_csv('/scratch/by8jj/stratified samples/ensemble model/file.csv')

In [4]:
len(df)

19493118

In [15]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(df.drop('label', axis = 1), df.label, test_size=0.2)

In [23]:
X_train['label'] = y_train

In [24]:
X_train

,log,ann,xgb,rf,label
3174338,0.690991,0.997195,0.698532,0.834990,0
17951744,0.194900,0.640728,0.337510,0.176110,0
10074204,0.000310,0.039187,0.116944,0.000000,0
3978769,0.004402,0.272602,0.114828,0.000392,0
6024148,0.003173,0.092988,0.118696,0.000000,0
11760816,0.939115,1.000000,0.753784,0.992513,1
17985303,0.000229,0.064191,0.117471,0.002288,0
7186515,0.015125,0.911867,0.416897,0.000316,0
5477260,0.003114,0.176869,0.115390,0.000000,0
12036889,0.000395,0.002271,0.118709,0.000000,0


In [25]:
df_mal = X_train[X_train['label'] == 1]
df_ben = X_train[X_train['label'] == 0].sample(frac = 1)[:len(df_mal)]
df_bal = pd.concat([df_mal, df_ben]).sample(frac = 1)

In [26]:
df_bal

,log,ann,xgb,rf,label
17344153,0.001322,0.007825,0.120707,0.000000,0
11411135,0.003129,0.013238,0.120952,0.000958,0
13483163,0.033849,0.243392,0.114756,0.013724,0
15892780,0.999947,1.000000,0.837873,0.999824,1
12261742,0.001892,0.081044,0.122400,0.000367,0
5651,0.999821,1.000000,0.845072,0.999997,1
18620826,0.002572,0.133906,0.114920,0.000119,0
2872144,0.222927,0.999978,0.423434,0.014220,0
5335563,0.997037,1.000000,0.800144,0.998902,1
5091356,0.694182,0.985524,0.723533,0.543562,1


In [27]:
y = df_bal.label.tolist()
X = np.matrix(df_bal.drop(labels = ['label'], axis = 1)).astype(np.float)
print(X.shape)

(1484072, 4)


In [28]:
model = models.Sequential()
model.add(Dense(2, input_dim=4, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0000002, amsgrad=False)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy',f1,recall,precision])

In [29]:
result = model.fit(X, y, epochs=20, batch_size=256, verbose=1, validation_split=0.3)

Train on 1038850 samples, validate on 445222 samples
Epoch 1/20
1038850/1038850 [==============================] - 5s 5us/step - loss: 0.6352 - acc: 0.6991 - f1: 0.6422 - recall: 0.5718 - precision: 0.7399 - val_loss: 0.5338 - val_acc: 0.9138 - val_f1: 0.9204 - val_recall: 0.9991 - val_precision: 0.8537
Epoch 2/20
1038850/1038850 [==============================] - 5s 5us/step - loss: 0.5221 - acc: 0.7494 - f1: 0.7040 - recall: 0.5991 - precision: 0.8567 - val_loss: 0.3794 - val_acc: 0.9434 - val_f1: 0.9462 - val_recall: 0.9961 - val_precision: 0.9014
Epoch 3/20
1038850/1038850 [==============================] - 5s 5us/step - loss: 0.4429 - acc: 0.7662 - f1: 0.7178 - recall: 0.5980 - precision: 0.9008 - val_loss: 0.2859 - val_acc: 0.9480 - val_f1: 0.9501 - val_recall: 0.9908 - val_precision: 0.9129
Epoch 4/20
1038850/1038850 [==============================] - 5s 5us/step - loss: 0.3951 - acc: 0.7680 - f1: 0.7187 - recall: 0.5961 - precision: 0.9079 - val_loss: 0.2345 - val_acc: 0.9500 -

In [30]:
y_pred = model.predict(np.matrix(X_test).astype(np.float))

In [31]:
y_pred = [x[0] for x in y_pred]

In [ ]:
temp = [x if x>0.5 else 0 for x in y_pred]

In [34]:
pred = pd.DataFrame({'test':y_test, 'pred': y_pred})

In [36]:
pred

,pred,test
2937210,0.023088,0
18849273,0.022539,0
15470253,0.085467,0
4074838,0.026822,0
6195558,0.023347,0
11576107,0.023304,0
18034584,0.041201,0
2808510,0.930900,1
12272186,0.022493,0
1126749,0.029622,0


In [42]:
temp = [1 if x>0.8 else 0 for x in y_pred]
cm= confusion_matrix(y_test, temp)
tn, fp, fn, tp = cm.ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
fpr = fp/(fp+ tn)
accuracy = (tp + tn)/(tn + tp + fn + fp)
F1 = 2 * (precision * recall) / (precision + recall)
print("precision:", precision*100)
print("recall:", recall*100)
print("false positive rate:", fpr*100)
print("accuracy",  accuracy*100)
print("F1-score", F1)

precision: 81.6491971891807
recall: 89.35790853042899
false positive rate: 1.0073213698881054
accuracy 98.5325078797032
F1-score 0.8532980501964162
